In [2]:
'''Purpose of this script is to clean and transform the first iteration of the EHS headcount data'''

import pandas as pd
import numpy as np

import re
import operator as op

In [8]:
# read in EHS/GS property list for property size, function, tier, location
prop = pd.read_excel('2017-09-11 EHS - Global Security Records of Interest.xlsx', header=7, skiprows=[0-7])
prop.head()

,File Client Id,Division,Region,System,EHS/Security Reporting,Security Contact,Tier 1-5,Country Region,Portfolio HUB,Building Country,...,Premises Description,Building Postal Code,Current Expiration Date,Rentable Area,Lease/Owned,Expense or Income,Lease Space Use,Client Position,Landlord Name,Tenant Name
0,UADUBBCT-LN-O,International & Telemundo,NBCU International,International TV Distribution,YES,Stephen Smith,5.0,EMEA,International,ARE,...,NaN,0000,2017-09-14,793.240000,LEASE,EXPENSE,OFFICE,TENANT,Business Central Towers FZ-LLC,NBC Universal International Television Distrib...
1,AUMEL005LU,International & Telemundo,NBCU International,International TV Distribution,YES,Stephen Smith,4.0,APAC,International,AUS,...,NaN,3185,2018-11-13,3003.131006,LEASE,EXPENSE,OFFICE,TENANT,A-One Properties Pty Ltd,Matchbox Pictures Pty Ltd
2,AUSYD001LU,International & Telemundo,NBCU International,Universal Filmed Entertainment,YES,Stephen Smith,2.0,APAC,International,AUS,...,Universal Pictures International Australia,2011,2018-01-14,16145.865625,LEASE,EXPENSE,OFFICE,SUBLESSEE,Woodben Pty Limited,Universal Pictures (Australasia) Pty. Ltd
3,AUSYD003LC,CNBC,CNBC International,CNBC International,YES,Stephen Smith,3.0,APAC,International,AUS,...,CNBC Asia Office Lease,2000,2017-12-31,204.514298,LEASE,EXPENSE,OFFICE,TENANT,Financial & Energy Exchange Limited,Business News (Asia) LLP (CNBC Asia Pacific)
4,ATVIE112LU,International & Telemundo,NBCU International,Universal Filmed Entertainment,YES,Stephen Smith,5.0,EMEA,International,AUT,...,Universal Pictures International Austria,1070,2017-06-30,3883.000000,LEASE,EXPENSE,OFFICE,TENANT,Orag Osterreichische Realitaten-Aktiengesellsc...,UPI Austria GMBH


In [ ]:
# get building area by country
group1 = prop.groupby(['Building Country']).sum().round(1).reset_index()
group1.sort_values('Rentable Area', ascending= False)

In [20]:
risk = pd.read_excel("Country Risk Ratings June 2017.xlsx")
risk.head()

,Country,NC4,Control Risks,Foreign Affairs Canada,FCO UK,State Department,NBCU,Tier,Comments
0,Afghanistan,Extreme,Extreme,Extreme,Extreme,Extreme,High,Tier 1 (All),"Tier 1 (All) As of 21 March, The Department of..."
1,Albania,Medium,Low,Low,Medium,Medium,Medium,NaN,NaN
2,Algeria,High,High,High,High,High,High,Tier 1 (Outside of Algiers)\n,"Tier 1 (Outside of Algiers) As of 13 December,..."
3,American Samoa,NaN,Low,Low,Low,NaN,Low,NaN,NaN
4,Andorra,NaN,Low,Low,NaN,NaN,Low,NaN,NaN


In [21]:
risk = risk.drop(['NC4','Control Risks','Foreign Affairs Canada','FCO UK','Tier','Comments'],axis=1)
risk.head()

,Country,State Department,NBCU
0,Afghanistan,Extreme,High
1,Albania,Medium,Medium
2,Algeria,High,High
3,American Samoa,NaN,Low
4,Andorra,NaN,Low


In [23]:
# read in dataframe of country and abbreviation
country = pd.read_excel('country_abb.xlsx', skiprows=[1], names=('Country','Alpha2','CountryAbb','NumCode'))
country.head()

,Country,Alpha2,CountryAbb,NumCode
0,Afghanistan,AF,AFG,4
1,Aland Islands,AX,ALA,248
2,Albania,AL,ALB,8
3,Algeria,DZ,DZA,12
4,American Samoa,AS,ASM,16


In [24]:
country = country.drop(['Alpha2','NumCode'],axis=1)
country.head()

,Country,CountryAbb
0,Afghanistan,AFG
1,Aland Islands,ALA
2,Albania,ALB
3,Algeria,DZA
4,American Samoa,ASM


In [25]:
risk = risk.join(country.set_index("Country"), on= "Country")

In [15]:
risk = risk.CountryAbb.str.replace('GBR','UK')

In [29]:
risk = risk.rename(columns={'NBCU':'NBC_RiskCategory'})
risk.head()

,Country,State Department,NBC_RiskCategory,CountryAbb
0,Afghanistan,Extreme,High,AFG
1,Albania,Medium,Medium,ALB
2,Algeria,High,High,DZA
3,American Samoa,NaN,Low,ASM
4,Andorra,NaN,Low,AND


In [27]:
# rename column Building Country to CountryAbb
prop = prop.rename(columns={'Building Country':'CountryAbb','NBCU':'NBC_RiskCategory'})
prop.head()

,File Client Id,Division,Region,System,EHS/Security Reporting,Security Contact,Tier 1-5,Country Region,Portfolio HUB,CountryAbb,...,Premises Description,Building Postal Code,Current Expiration Date,Rentable Area,Lease/Owned,Expense or Income,Lease Space Use,Client Position,Landlord Name,Tenant Name
0,UADUBBCT-LN-O,International & Telemundo,NBCU International,International TV Distribution,YES,Stephen Smith,5.0,EMEA,International,ARE,...,NaN,0000,2017-09-14,793.240000,LEASE,EXPENSE,OFFICE,TENANT,Business Central Towers FZ-LLC,NBC Universal International Television Distrib...
1,AUMEL005LU,International & Telemundo,NBCU International,International TV Distribution,YES,Stephen Smith,4.0,APAC,International,AUS,...,NaN,3185,2018-11-13,3003.131006,LEASE,EXPENSE,OFFICE,TENANT,A-One Properties Pty Ltd,Matchbox Pictures Pty Ltd
2,AUSYD001LU,International & Telemundo,NBCU International,Universal Filmed Entertainment,YES,Stephen Smith,2.0,APAC,International,AUS,...,Universal Pictures International Australia,2011,2018-01-14,16145.865625,LEASE,EXPENSE,OFFICE,SUBLESSEE,Woodben Pty Limited,Universal Pictures (Australasia) Pty. Ltd
3,AUSYD003LC,CNBC,CNBC International,CNBC International,YES,Stephen Smith,3.0,APAC,International,AUS,...,CNBC Asia Office Lease,2000,2017-12-31,204.514298,LEASE,EXPENSE,OFFICE,TENANT,Financial & Energy Exchange Limited,Business News (Asia) LLP (CNBC Asia Pacific)
4,ATVIE112LU,International & Telemundo,NBCU International,Universal Filmed Entertainment,YES,Stephen Smith,5.0,EMEA,International,AUT,...,Universal Pictures International Austria,1070,2017-06-30,3883.000000,LEASE,EXPENSE,OFFICE,TENANT,Orag Osterreichische Realitaten-Aktiengesellsc...,UPI Austria GMBH


In [30]:
prop = prop.join(risk.set_index("CountryAbb"),on='CountryAbb')

In [31]:
prop.head()

,File Client Id,Division,Region,System,EHS/Security Reporting,Security Contact,Tier 1-5,Country Region,Portfolio HUB,CountryAbb,...,Rentable Area,Lease/Owned,Expense or Income,Lease Space Use,Client Position,Landlord Name,Tenant Name,Country,State Department,NBC_RiskCategory
0,UADUBBCT-LN-O,International & Telemundo,NBCU International,International TV Distribution,YES,Stephen Smith,5.0,EMEA,International,ARE,...,793.240000,LEASE,EXPENSE,OFFICE,TENANT,Business Central Towers FZ-LLC,NBC Universal International Television Distrib...,United Arab Emirates,Medium,Medium
1,AUMEL005LU,International & Telemundo,NBCU International,International TV Distribution,YES,Stephen Smith,4.0,APAC,International,AUS,...,3003.131006,LEASE,EXPENSE,OFFICE,TENANT,A-One Properties Pty Ltd,Matchbox Pictures Pty Ltd,Australia,Low,Low
2,AUSYD001LU,International & Telemundo,NBCU International,Universal Filmed Entertainment,YES,Stephen Smith,2.0,APAC,International,AUS,...,16145.865625,LEASE,EXPENSE,OFFICE,SUBLESSEE,Woodben Pty Limited,Universal Pictures (Australasia) Pty. Ltd,Australia,Low,Low
3,AUSYD003LC,CNBC,CNBC International,CNBC International,YES,Stephen Smith,3.0,APAC,International,AUS,...,204.514298,LEASE,EXPENSE,OFFICE,TENANT,Financial & Energy Exchange Limited,Business News (Asia) LLP (CNBC Asia Pacific),Australia,Low,Low
4,ATVIE112LU,International & Telemundo,NBCU International,Universal Filmed Entertainment,YES,Stephen Smith,5.0,EMEA,International,AUT,...,3883.000000,LEASE,EXPENSE,OFFICE,TENANT,Orag Osterreichische Realitaten-Aktiengesellsc...,UPI Austria GMBH,Austria,Low,Low


In [32]:
# count of null rows per column
prop.isnull().sum()

File Client Id              56
Division                   140
Region                     140
System                     140
EHS/Security Reporting     140
Security Contact           151
Tier 1-5                   151
Country Region             140
Portfolio HUB              140
CountryAbb                 140
Building State/Province    140
Building City              140
Building Address 1         140
Premises Description       242
Building Postal Code       140
Current Expiration Date    159
Rentable Area              140
Lease/Owned                140
Expense or Income          140
Lease Space Use            140
Client Position            140
Landlord Name              145
Tenant Name                164
Country                      5
State  Department          133
NBC_RiskCategory             5
dtype: int64

In [6]:
prop.to_excel('Incident_Model_data_100317.xlsx',index=False)

In [2]:
prop = pd.read_excel('Incident_Model_data_100317.xlsx')
prop.head()

,File Client Id,Division,Region,System,EHS/Security Reporting,Security Contact,Tier 1-5,Country Region,Portfolio HUB,CountryAbb,...,Rentable Area,Lease/Owned,Expense or Income,Lease Space Use,Client Position,Landlord Name,Tenant Name,Country,State Department,NBC_RiskCategory
0,UADUBBCT-LN-O,International & Telemundo,NBCU International,International TV Distribution,YES,Stephen Smith,5.0,EMEA,International,ARE,...,793.240000,LEASE,EXPENSE,OFFICE,TENANT,Business Central Towers FZ-LLC,NBC Universal International Television Distrib...,United Arab Emirates,Medium,Medium
1,AUMEL005LU,International & Telemundo,NBCU International,International TV Distribution,YES,Stephen Smith,4.0,APAC,International,AUS,...,3003.131006,LEASE,EXPENSE,OFFICE,TENANT,A-One Properties Pty Ltd,Matchbox Pictures Pty Ltd,Australia,Low,Low
2,AUSYD001LU,International & Telemundo,NBCU International,Universal Filmed Entertainment,YES,Stephen Smith,2.0,APAC,International,AUS,...,16145.865625,LEASE,EXPENSE,OFFICE,SUBLESSEE,Woodben Pty Limited,Universal Pictures (Australasia) Pty. Ltd,Australia,Low,Low
3,AUSYD003LC,CNBC,CNBC International,CNBC International,YES,Stephen Smith,3.0,APAC,International,AUS,...,204.514298,LEASE,EXPENSE,OFFICE,TENANT,Financial & Energy Exchange Limited,Business News (Asia) LLP (CNBC Asia Pacific),Australia,Low,Low
4,ATVIE112LU,International & Telemundo,NBCU International,Universal Filmed Entertainment,YES,Stephen Smith,5.0,EMEA,International,AUT,...,3883.000000,LEASE,EXPENSE,OFFICE,TENANT,Orag Osterreichische Realitaten-Aktiengesellsc...,UPI Austria GMBH,Austria,Low,Low


In [5]:
hr = pd.read_excel("NBCUniversal Headcount-OctHC 2017.xlsx")
hr.head()

,uid,pernrid,hrfullname,givenname,sn,hrmiddlename,hrknownas,hrprenameadjunct,hrsuffix,hrtitle,...,suborgsegment,suphierarchy,loginid,ccworkemail,hrpositionid,tandeapprover,exceptionapprover,delegateapprover,employeesubgroup,agile1access
0,700679491,679491.0,"Martinowsky, John",John,Martinowsky,J,NaN,NaN,NaN,NaN,...,NaN,6042002382061010354282915786357604693916856787...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,604000096,604000096.0,"De Cardenas,Juan Carlos",Juan,De Cardenas,Carlos,NaN,NaN,NaN,NaN,...,NaN,604000327604000000642653784856074396448587776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,206553188,10128619.0,"Czaczka, Maja",Maja,Czaczka,NaN,NaN,NaN,NaN,Miss,...,International Corporate,6042002382061010354282915786357604693916856787...,NaN,NaN,51405179.0,NaN,NaN,NaN,Fixed Term Contract,NaN
3,206099828,8800277.0,"Howard, Lynn",Lynn,Howard,C,NaN,NaN,NaN,NaN,...,NBC Sports Regional Networks,6042002382061010354282915786357604693916856787...,NaN,NaN,86099828.0,NaN,NaN,NaN,Daily Hires,NaN
4,703057331,3057331.0,"Thomas, Shannon",Shannon,Thomas,C,NaN,NaN,NaN,NaN,...,NaN,6042002382061010354282915786357604693916856787...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
hr.columns

NameError: name 'hr' is not defined

In [8]:
hr.groupby(['address','country','city','state']).count().to_excel('Headcount_address.xlsx')

In [72]:
adv = pd.read_excel("GRIC_Advisories_100217 copy.xlsx")

#drop City_Mapped column to have clean start to run loops
adv.drop('City_Mapped', inplace=True, axis=1)
### fill blank rows with a space so they can be parsed
adv.Advisory = adv.Advisory.fillna("")
adv.head()

,#_of_Advisories_Sent,Advisory,AdvisoryNotes,Brand/Reputation,Business Operations,City,Country,Date,Impact_Score,Incident Type,Personal Safety,State
0,1.0,Medical Condition at 30 Rock,NaN,0.0,0.0,New York,United States,2015-07-01,4.0,Medical Incident,4.0,New York
1,1.0,Possibility of violent attacks throughout Egypt,NaN,0.0,2.0,NaN,Egypt,2015-07-01,5.0,Violent Criminal Activity,3.0,NaN
2,2.0,Greek Protests,NaN,3.0,3.0,NaN,Greece,2015-07-01,9.0,Protest,3.0,NaN
3,1.0,Japan Bullet Train Fire,NaN,0.0,0.0,NaN,Japan,2015-07-01,4.0,Transportation Incident,4.0,NaN
4,1.0,Navy Yark Lockdown,NaN,5.0,5.0,New York,United States,2015-07-02,15.0,Active Shooter/Lock Down,5.0,New York


In [12]:
#url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'

In [36]:
# df of us cities
us_city = pd.read_excel('UScities.xlsx')

In [33]:
### removes the brackets and numbers from city names in the cities file
us_city.City = cities.City.str.replace(r"\[.*\]","")

In [34]:
us_city['Country'] = 'United States' # assign Country to US cities
us_city = cities.rename(columns={'State[5]':'State'}) # rename column
us_city.to_excel('UScities.xlsx')
us_city.head()

,2016rank,City,State,2016estimate,2010Census,Change,2016 land area,2016 population density,Location,Unnamed: 9,Unnamed: 10,Country
0,1,New York,New York,8537673,8175133,7000443466791304800♠+4.43%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N73°56′19″W/40.6635°N 73.9387°W/40.66...,United States
1,2,Los Angeles,California,3976322,3792621,7000484364243092050♠+4.84%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N118°24′39″W/34.0194°N 118.4108°W/34....,United States
2,3,Chicago,Illinois,2704958,2695598,6999347232784710470♠+0.35%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N87°40′54″W/41.8376°N 87.6818°W/41.83...,United States
3,4,Houston,Texas,2303482,2100263,7000967588344888229♠+9.68%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N95°23′27″W/29.7866°N 95.3909°W/29.78...,United States
4,5,Phoenix,Arizona,1615017,1445632,7001117170206525590♠+11.72%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N112°05′24″W/33.5722°N 112.0901°W/33....,United States


In [42]:
us_city = pd.read_excel('UScities.xlsx')

int_city = pd.read_excel("WorldCities.xlsx")
int_city.head()

,City,Country,Population-City Proper,Population-Metropolitan area,Population-Urban area
0,Shanghai,China,"24,256,800[8]","34,750,000[9]","23,416,000[a]"
1,Beijing,China,"21,516,000[10]","24,900,000[11]",21009000
2,Delhi,India,"16,787,941[12]",24998000,"21,753,486[13]"
3,Lagos,Nigeria,"16,060,303[b]","21,000,000[16]",13123000
4,Karachi,Pakistan,"14,910,352[17]",NaN,NaN


In [17]:
df_cit = pd.concat([us_city,int_city],axis=0,join='outer')

In [64]:
### some of the rows in the Advisory column are stored in weird formats, 
# code below unifies the way each row is displayed as text
#adv['Advisory'] = adv['Advisory'].apply(lambda x: x.decode('utf-8').strip())

In [150]:
''' trying to run the below code on a csv file. Formatting issues with Advisory col'''
#df = pd.read_csv('GRIC_Advisories_100217 copy.csv')
""" looks like some of the rows in the Advisory column are stored in weird formats, 
#so the code below unifies the way each row is displayed as text"""
#adv.to_csv('teswaltesd.csv')
#df['Advisory'] = df['Advisory'].apply(lambda x: x.decode('utf-8').strip())

' looks like some of the rows in the Advisory column are stored in weird formats, \n#so the code below unifies the way each row is displayed as text'

In [5]:
adv['City_Mapped'] = ''

"""This function will loop through each row in the Advisory column
and see if the current city being looped through is contained within that row. The try/except part 
is mainly because some of the text in the Advisory column is causing Encoding Errors.. that usually 
means it's just in a weird format"""

def match_city_to_advisory_details(row):
    try:
        if re.search('{city}'.format(city = city),row.Advisory) is not None:
            row['City_Mapped'] = city
        else:
            pass
    except UnicodeEncodeError:
        pass
    return row

In [73]:
"""This is a for loop that applies the above function to the dataset FOR each unique city in the df_cit.City column"""
"""to avoid the city Cali being associated with California, create an international and a national df"""
for city in us_city['City'].unique():
    adv = adv.apply(lambda row: match_city_to_advisory_details(row), axis = 1)

In [74]:
for city in int_city['City'].unique():
    adv = adv.apply(lambda row: match_city_to_advisory_details(row), axis=1)

In [75]:
# zero out rows where Cali is the City bc Cali, Columbia should not be mapped to California
adv.City = adv.City.str.replace('California',"")

In [76]:
adv.City = adv.City.fillna(adv.City_Mapped)
adv.City = adv.City.str.replace('Cali',"")
#adv.City = adv.City.str.replace('California',"")
adv.City = adv.City.str.replace('Universal City',"")

In [67]:
#for i in adv.City:
#    if adv.City is not None:
#        pass
#    if adv.Country[i] != "California":
#        adv.City.fillna(adv.City_Mapped)
        

In [89]:
adv.to_excel('GRIC_Advisories_101017.xlsx',index=False)

In [5]:
adv = pd.read_excel('GRIC_Advisories_101217.xlsx')
adv.head()

,Date,Advisory,Incident Type,Country,City,State,#_of_Advisories_Sent,AdvisoryNotes,Personal Safety,Business Operations,Brand/Reputation,Impact_Score
0,2015-07-01,Medical Condition at 30 Rock,Medical Incident,United States,New York,New York,1.0,NaN,4.0,0.0,0.0,4.0
1,2015-07-01,Possibility of violent attacks throughout Egypt,Violent Criminal Activity,Egypt,NaN,NaN,1.0,NaN,3.0,2.0,0.0,5.0
2,2015-07-01,Greek Protests,Protest,Greece,NaN,NaN,2.0,NaN,3.0,3.0,3.0,9.0
3,2015-07-01,Japan Bullet Train Fire,Transportation Incident,Japan,NaN,NaN,1.0,NaN,4.0,0.0,0.0,4.0
4,2015-07-02,Navy Yark Lockdown,Active Shooter/Lock Down,United States,New York,New York,1.0,NaN,5.0,5.0,5.0,15.0


In [10]:
sorted(adv.City.unique())

['',
 u'Alexandria',
 u'Alpine',
 u'Amarillo',
 u'Amsterdam',
 u'Ankara',
 u'Antioch',
 u'Arlington',
 u'Athens',
 u'Atlanta',
 u'Auckland',
 u'Austin',
 u'Azusa',
 u'Baghdad',
 u'Baltimore',
 u'Bangkok',
 u'Barcelona',
 u'Batangas',
 u'Baton Rouge',
 u'Beijing',
 u'Belfast',
 u'Berkeley',
 u'Berlin',
 u'Bethesda',
 u'Boston',
 u'Boulder',
 u'Bridgeport',
 u'Bridgewater Plaza',
 u'Budapest',
 u'Buenos Aires',
 u'Burbank',
 u'Cairo',
 u'Calgary',
 u'Cancun',
 u'Cannonball',
 u'Cape Town',
 u'Caracas',
 u'Charleston',
 u'Charlotte',
 u'Charlottesville',
 u'Chattanooga',
 u'Chicago',
 u'Cleveland',
 u'Cologne',
 u'Colonie',
 u'Colorado',
 u'Colorado Springs',
 u'Columbia',
 u'Columbus',
 u'Copenhagen',
 u'Coral Gables',
 u'Dakar',
 u'Dakota',
 u'Dallas',
 u'Denver',
 u'Detroit',
 u'Dhaka',
 u'Dubai',
 u'Dublin',
 u'Elizabeth',
 u'Englewood Cliffs',
 u'Erbil',
 u'Esperanza',
 u'Fairfield',
 u'Ferguson',
 u'Florida',
 u'Fort Campbell',
 u'Fort Lauderdale',
 u'Fort Lee',
 u'Fort Wayne',
 u'F

In [84]:
adv.City = adv.City.str.replace('New York City', "New York")
adv.City = adv.City.str.replace('New York ', "New York")

In [86]:
adv.City = adv.City.str.replace('Washington ', "Washington D.C.")

In [9]:
# fill null values so below map can work
adv.City = adv.City.fillna("")
# remove any trailing spaces
adv.City = adv.City.map(lambda x: x.strip())